Universidade Federal de Minas Gerais

Departamento de Ciência da Computação

Bacharelado em Ciência da Computação

Introdução a Computação Visual – 2017/1

Professor: Erickson R. Nascimento e William Robson Schwartz

# Trabalho Pŕatico 2 - Realidade Aumentada

### Alison de Oliveira Souza - 2012049316

## Etapa 1: Calibração da câmera

Para calibrar a câmera usei o código calibration.cpp disponibilizado na pasta samples/cpp do OpenCV 3.1.0.

Compilei tal código da seguinte maneira:

g++ -Iusr/include/opencv2 calibration.cpp $(pkg-config opencv --libs) -o calibration

Após compilar, executei o binário calibration da seguinte maneira:

./calibration -w=6 -h=8 -s=0.03 -n=8 -V entrada.avi -o=intrinsic.yml -zt

Com isso, foi gerado o arquivo intrinsic.yml com a seguinte saída:

In [ ]:
%YAML:1.0
calibration_time: "ter 30 mai 2017 14:33:09 -03"
image_width: 640
image_height: 480
board_width: 6
board_height: 8
square_size: 2.9999999329447746e-02
aspectRatio: 1.
flags: 10
camera_matrix: !!opencv-matrix
   rows: 3
   cols: 3
   dt: d
   data: [ 5.8841583430397554e+02, 0., 3.0732260409727314e+02, 0.,
       5.8841583430397554e+02, 2.2350170630520270e+02, 0., 0., 1. ]
distortion_coefficients: !!opencv-matrix
   rows: 5
   cols: 1
   dt: d
   data: [ 4.3817979965505918e-02, 2.8013203741313847e-01, 0., 0.,
       1.3472499999780172e+01 ]
avg_reprojection_error: 2.5354900956128007e-01


## Etapa 2: Detecção e Localização dos alvos
Para essa etapa utilizei os conceitos e funções pré-definidas no link:

https://rdmilligan.wordpress.com/2015/07/19/glyph-recognition-using-opencv-and-python/

Essas funções ajudam na detecção de padrões binários.

In [ ]:
# Importando bibliotecas necessárias para o trabalho.
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt

###############################################################
##                     FUNÇÕES AUXILIARES                    ##
###############################################################
def order_points(points):
 
    s = points.sum(axis=1)
    diff = np.diff(points, axis=1)
     
    ordered_points = np.zeros((4,2), dtype="float32")
 
    ordered_points[0] = points[np.argmin(s)]
    ordered_points[2] = points[np.argmax(s)]
    ordered_points[1] = points[np.argmin(diff)]
    ordered_points[3] = points[np.argmax(diff)]
 
    return ordered_points
 
def max_width_height(points):
 
    (tl, tr, br, bl) = points
 
    top_width = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    bottom_width = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    max_width = max(int(top_width), int(bottom_width))
 
    left_height = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    right_height = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    max_height = max(int(left_height), int(right_height))
 
    return (max_width,max_height)
 
def topdown_points(max_width, max_height):
    return np.array([
        [0, 0],
        [max_width-1, 0],
        [max_width-1, max_height-1],
        [0, max_height-1]], dtype="float32")
 
def get_topdown_quad(image, src):
 
    # src and dst points
    src = order_points(src)
 
    (max_width,max_height) = max_width_height(src)
    dst = topdown_points(max_width, max_height)
  
    # warp perspective
    matrix = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(image, matrix, max_width_height(src))
 
    # return top-down quad
    return warped

def get_glyph_pattern(image, black_threshold, white_threshold):
 
    # collect pixel from each cell (left to right, top to bottom)
    cells = []
     
    cell_half_width = int(round(image.shape[1] / 14.0))
    cell_half_height = int(round(image.shape[0] / 14.0))
 
    row1 = cell_half_height*3
    row2 = cell_half_height*5
    row3 = cell_half_height*7
    row4 = cell_half_height*9
    row5 = cell_half_height*11
    col1 = cell_half_width*3
    col2 = cell_half_width*5
    col3 = cell_half_width*7
    col4 = cell_half_width*9
    col5 = cell_half_width*11
 
    cells.append(image[row1, col1])
    cells.append(image[row1, col2])
    cells.append(image[row1, col3])
    cells.append(image[row1, col4])
    cells.append(image[row1, col5])
    cells.append(image[row2, col1])
    cells.append(image[row2, col2])
    cells.append(image[row2, col3])
    cells.append(image[row2, col4])
    cells.append(image[row2, col5])
    cells.append(image[row3, col1])
    cells.append(image[row3, col2])
    cells.append(image[row3, col3])
    cells.append(image[row3, col4])
    cells.append(image[row3, col5])
    cells.append(image[row4, col1])
    cells.append(image[row4, col2])
    cells.append(image[row4, col3])
    cells.append(image[row4, col4])
    cells.append(image[row4, col5])
    cells.append(image[row5, col1])
    cells.append(image[row5, col2])
    cells.append(image[row5, col3])
    cells.append(image[row5, col4])
    cells.append(image[row5, col5])
 
    # threshold pixels to either black or white
    for idx, val in enumerate(cells):
        if val < black_threshold:
            cells[idx] = 0
        elif val > white_threshold:
            cells[idx] = 1
        else:
            return None
 
    return cells
 
def resize_image(image, new_size):
    ratio = new_size / image.shape[1]
    return cv2.resize(image,(int(new_size),int(image.shape[0]*ratio)))
 
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, rotation_matrix, (w, h))

###############################################################
###############################################################
###############################################################

SHAPE_RESIZE = 100.0
BLACK_THRESHOLD = 127
WHITE_THRESHOLD = 128

# Lendo imagem alvo, ou seja, padrão que será buscado no vídeo.
img = cv2.imread('alvo.jpg')

# Aplicando imagem em escala de cinza e eliminando ruídos borrando a imagem.
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlurred = cv2.GaussianBlur(imgGray, (7, 7), 0)

# Salva o padrão a ser buscado na variável img_pattern
img_pattern = get_glyph_pattern(imgBlurred, BLACK_THRESHOLD, WHITE_THRESHOLD)

# Abrindo o vídeo de entrada.
camera = cv2.VideoCapture('entrada.avi')

# Cria vídeo de saída.
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

# Loop para iterar sobre cada frame do vídeo.
while True:
    # Lendo o frame atual e salvando o status da leitura na flag readed.
    (readed, frame) = camera.read()

    # Verificado se há mais frames no vídeo. Caso tenha chegado
    # ao fim do vídeo, encerro o loop de iteração sobre frames.
    if not readed:
        break
 
    # Convertendo o frame em grayscale, borrando o frame para remover excesso de
    # ruídos (e pequenas bordas), e detectando bordas.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(blurred, 100, 200)
    
    # Encontra os contornos nas imagens de bordas.
    (image, countours, _) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    countours = sorted(countours, key=cv2.contourArea, reverse=True)[:10]

    # Loop de iteração sobre os contornos.
    for c in countours:
        # Escolhe os contornos com mais de 40 pontos, eliminando bordas pequenas.
        if len(c) > 40:
            # Aproximação Poligonal dos contornos.
            perimeter = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.01 * perimeter, True)
            
            # Aproveitando apenas contornos que são aproximadamente retangulares.
            if len(approx) == 4:
                
                # Função que rotaciona os quadrilateros encontrados para facilitar
                # identificação dos alvos.
                topdown_quad = get_topdown_quad(blurred, approx.reshape(4,2))
                
                # Redimensiona o objeto e verifica se há um pixel escuro dentro da borda.
                # Se não houver, nenhum então descartamos esse objeto, pois nosso alvo tem
                # pontos pretos. Isso basicamente descarta os quadrados brancos dentro do padrão.
                resized_shape = resize_image(topdown_quad, SHAPE_RESIZE)
                if resized_shape[7, 7] > BLACK_THRESHOLD:
                    continue
                
                # Flag que indica que encontramos o alvo.
                glyph_found = False
                
                # Realiza 4 buscas de padrão, uma de cada lado do alvo, rotacionando ele 90° por vez.
                for i in range(4):
                    glyph_pattern = get_glyph_pattern(resized_shape, BLACK_THRESHOLD, WHITE_THRESHOLD)
                    
                    if glyph_pattern == img_pattern:
                        glyph_found = True
                        break
                        
                    resized_shape = rotate_image(resized_shape, 90)
                
                # Se encontrou o padrão.
                if glyph_found:
                    # Hora de desenhar o quadrado sobre o alvo
                    # e por o pikachu.
                    #print "ENTROU NO IF"
                    # LUGAR CERTO PARA O DRAWCONTOURS, PORÉM NÃO FUNCIONA!!!
                    #cv2.drawContours(frame, approx, -1, (0, 0, 255), 4)
                    break
                
                # Lugar errado para a drawContours. Aqui ela marca qualquer polígono.
                cv2.drawContours(frame, [approx], -1, (0, 0, 255), 4)
                cv2.imshow('Frame', frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                
                cv2.imshow('Bordas', image)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
    out.write(frame)

camera.release()
out.release()
cv2.destroyAllWindows()

Não consegui realizar a marcação correta em volta dos padrões. Dentro do if que verifica se o padrão foi encontrado, há uma chamada para a função drawContours que não funciona. Fiz um teste colocando um print dentro do if, para verificar se o fluxo de execução está realmente passando por lá, e a frase ENTROU NO IF é printada, mostrando que o fluxo de execução passa pelo if, várias vezes inclusive. Mas por algum motivo, que não conseguir descobrir qual, o contorno dos padrões não é exibido na tela.

Ao colocar a mesma função drawContours do lado de fora do if, a função desenha os contornos, mas não apenas nos padrões, e sim em qualquer polígono com aproximação com 4 pontos, incluindo alguns quadrados do tabuleiro. Por esse motivo, decedi enviar o  vídeo que marca todos os contornos, mesmo que errados. Caso queira ver a execução que deveria funcionar, basta comentar a linha onde há a função cv2.drawContours indicada como errada e descomentar a linha marcada como correta logo acima.

Por esse e outros motivos não consegui ir adiante. Instalar e configurar o OpenCV e Jupyter-Notebook para fazer esse trabalho foi algo extremamente trabalhoso. Tive que compilar o OpenCV no Linux pelo menos 3 vezes, isso depois de ter instalado pelo menos 2 versões diferentes (3.1 e 3.2) no Windows, ambas com problemas ao ler o vídeo, por falta de integração com o ffmpeg.
